In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import re

In [4]:
def scroll_down(driver):
    # 페이지 맨 아래로 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # 스크롤 후 잠시 대기

def get_news_info(driver, query):
    # 기사 정보 추출
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    news_items = soup.find_all('div', class_='news_area')
    
    news_info = []
    for item in news_items:
        # 기사 제목 추출
        title_elem = item.find('a', class_='news_tit')
        title = title_elem.text.strip() if title_elem else None
        
        # 제목에 검색어가 포함되어 있지 않은 경우 크롤링하지 않음
        if not title or query not in title:
            continue

        # 기사 날짜 추출
        date_elems = item.find_all('span', class_='info')
        date = None
        for date_elem in date_elems:
            if date_elem.find('i', class_='spnew ico_paper'):
                continue  # i 태그가 있는 경우 건너뛰기
            # 날짜 형식에 맞는 텍스트 확인
            if re.match(r'\d{4}\.\d{2}\.\d{2}', date_elem.text.strip()):
                date = date_elem.text.strip()
                break
        
        if date:
            news_info.append({'제목': title, '날짜': date})
    
    return news_info

# 검색어 입력
query = input("뉴스 검색어를 입력하세요: ")

# 날짜 설정
start_date = datetime(2019, 3, 1)
# start_date = datetime(2023, 12, 1)

end_date = datetime(2024, 1, 31)

# WebDriver로 Chrome 브라우저 열기
driver = webdriver.Chrome()

# 빈 리스트 생성
data = []

# 각 달마다 검색 수행
current_date = start_date
while current_date <= end_date:
    # 현재 달의 시작일과 종료일 계산
    next_month = current_date + timedelta(days=31)
    next_month = next_month.replace(day=1)  # 다음 달의 1일로 설정
    end_of_month = next_month - timedelta(days=1)  # 다음 달의 마지막 날짜로 설정
    
    # 네이버 뉴스 검색 페이지로 이동
    driver.get(f"https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={current_date.strftime('%Y.%m.%d')}&de={end_of_month.strftime('%Y.%m.%d')}")
    
    # 스크롤 다운을 100번 반복하여 데이터 가져오기 - 달 별로 1000개 
    for i in range(150): 
        scroll_down(driver)
        
    # 스크롤 다운이 완료된 후 데이터 가져오기
    news_info = get_news_info(driver, query)
    if news_info:
        for item in news_info:
            # 데이터 리스트에 추가
            data.append({'제목': item['제목'], '날짜': item['날짜']})
    
    # 다음 달로 이동
    current_date = next_month

# 브라우저 닫기
driver.quit()

# 데이터프레임 생성
df = pd.DataFrame(data)

# '제목' 열에서 중복 제거
df = df.drop_duplicates(subset='제목').reset_index(drop=True)

# 데이터프레임 출력
print(df)


뉴스 검색어를 입력하세요: KB금융
                                                     제목           날짜
0                          윤종규 KB금융 회장 "친환경 기업 금융지원 확대"  2019.03.31.
1               KB금융 주총서 "주가 낮다" 성토…KB "과감한 M&A 실행"(종합)  2019.03.27.
2                        '주사위는 던져졌다' KB금융 부동산 투자 '광폭행보'  2019.03.29.
3                              투기자본감시센터, KB금융 주총장 입장 무산  2019.03.27.
4                금융사 CEO ‘사회적 책임’ 강조···KB금융 주주들 “실적 저조”  2019.03.27.
...                                                 ...          ...
27882                   KB금융그룹, 2024년 상반기 그룹 경영진 워크숍 개최  2024.01.08.
27883                    동계스포츠 후원자 KB금융, 피겨 꿈나무에 장학금 전달  2024.01.08.
27884  소상공인에 진심인 KB금융·국민은행... 양종희 회장의 '상생 경영' [줌人CEO...  2024.01.18.
27885       양종희 KB금융 회장 "'경쟁과 생존'에서 '상생과 공존으로' 패러다임 전환"  2024.01.02.
27886          [신년메시지] 양종희 KB금융 회장 "'위기를 기회로'…상생·공존 필요"  2024.01.02.

[27887 rows x 2 columns]


In [6]:
df.to_csv('KB금융.csv', index=False)

In [5]:
df

,제목,날짜
0,"윤종규 KB금융 회장 ""친환경 기업 금융지원 확대""",2019.03.31.
1,"KB금융 주총서 ""주가 낮다"" 성토…KB ""과감한 M&A 실행""(종합)",2019.03.27.
2,'주사위는 던져졌다' KB금융 부동산 투자 '광폭행보',2019.03.29.
3,"투기자본감시센터, KB금융 주총장 입장 무산",2019.03.27.
4,금융사 CEO ‘사회적 책임’ 강조···KB금융 주주들 “실적 저조”,2019.03.27.
...,...,...
27882,"KB금융그룹, 2024년 상반기 그룹 경영진 워크숍 개최",2024.01.08.
27883,"동계스포츠 후원자 KB금융, 피겨 꿈나무에 장학금 전달",2024.01.08.
27884,소상공인에 진심인 KB금융·국민은행... 양종희 회장의 '상생 경영' [줌人CEO...,2024.01.18.
27885,"양종희 KB금융 회장 ""'경쟁과 생존'에서 '상생과 공존으로' 패러다임 전환""",2024.01.02.


In [31]:
titles = [key['제목'] for key in news_info]
dates = [item['날짜'] for item in news_info]
titles

['셀트리온 "바이오 스타트업 4개사 육성·지원…동반성장"',
 '서울시-셀트리온, 바이오·의료 창업기업 성장 지원',
 '"2조 美 악템라 시장 열렸다"… 셀트리온 품목허가 도전',
 '셀트리온-서울시, 바이오벤처 육성 업무협약',
 '셀트리온, 악템라 시밀러 CT-P47 美 품목허가 신청 제출',
 "셀트리온, 서울바이오허브와 '오픈 이노베이션 프로그램' 본격화",
 '셀트리온, 악템라 바이오시밀러 美 FDA에 품목허가 신청',
 "셀트리온, 악템라 바이오시밀러 'CT-P47' 미국 품목허가 신청",
 '셀트리온 “악템라 바이오시밀러 美 품목허가 신청”',
 '셀트리온, 악템라 바이오시밀러 美 품목 허가 신청',
 "셀트리온, 美 FDA에 '악템라' 바이오시밀러 품목허가 신청",
 '셀트리온-서울바이오허브, 오픈 이노베이션 프로그램 본격화',
 '셀트리온, 자가면역질환 치료제 악템라 시밀러 ‘CT-P47’ 美 품목허가 신청',
 '셀트리온, 자가면역질환 치료제 美 품목허가 신청',
 '셀트리온-서울바이오허브, 오픈 이노베이션 프로그램 본격화',
 '셀트리온, 서울바이오허브와 오픈 이노베이션 프로그램 본격화',
 '셀트리온, 악템라 시밀러 미국 품목허가 신청',
 "셀트리온 '오픈 이노' 프로그램서 바이오 스타트업 4개사 선발",
 '셀트리온, 악템라 바이오시밀러 CT-P47 美품목허가 신청',
 '셀트리온-서울바이오허브 "오픈 이노베이션 프로그램 본격화"',
 '셀트리온, 미국 FDA에 류마티스 관절염 바이오시밀러 품목허가 신청',
 '셀트리온, 4兆 시장 ‘악템라’ 바이오시밀러 글로벌 출사표',
 '셀트리온, 자가면역질환 시밀러 美 허가 신청',
 "셀트리온, 악템라 바이오시밀러 'CT-P47' 美 품목허가 신청",
 '셀트리온, 악템라 바이오시밀러 CT-P47 미국 품목허가 신청',
 '셀트리온, 악템라 바이오시밀러 미국 품목허가 신청',
 '셀트리온-서울바이오허브, 오픈 이노베이션 프로그램 가속도... 스타트업 4개...',
 '셀트리온, 악템라 바이